In [1]:
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
secrets_file = open("../secrets/secrets.txt","r")
string = secrets_file.read()
secrets_file.close()

In [3]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [4]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [5]:
def spotiwrap(category, search, limit, off):
    # with open('spotilogger.log', 'r') as lg:
    #     prevlog = lg.read()
        
    logfile = open('spotilogger.log', 'a')
    
    try:
        tracks = sp.search(q=f'{category}:{search}', type='track', limit=limit, offset=off)['tracks']['items']
    except:
        logfile.write(f'\t### error recieving data from the API ###\n')
    
    if len(tracks) == 0:
        logfile.write(f'\t### no entries ###\n')
        return

    c = 0
    for track in tracks:
        song = {}
        features = sp.audio_features(track['uri'])

        song['track_name'] = track['name']
        song[category] = search
        song['id'] = track['id']
        song['preview'] = track['preview_url']
        song_pd = pd.DataFrame([song])
        song_pd = pd.concat([song_pd, pd.json_normalize(features[0])], axis=1)

        if c == 0:
            songs = song_pd
        else:
            songs = pd.concat([songs, song_pd], axis=0)
        c += 1
    logfile.write(f'\t{c} songs recieved\n')
    logfile.close()
    return songs

In [6]:
def filldb(category, searchlist, limit=50, offset=0, savepoints=100):
    x = 0
    count = len(searchlist)
    if count > 1:
        what = count
    else:
        what = searchlist[0]

    for search in searchlist:

        with open('spotilogger.log', 'a') as logfile:
            logfile.write(f'{category}{x:6.0f}: {search}\n')

        if x == 0:
            songdb = spotiwrap(category, search, limit, offset)
        else:
            songdb = pd.concat([songdb, spotiwrap(category, search, limit, offset)])
        x += 1

        if x % savepoints == 0:
            songdb.to_csv(f'YOUR_SONG_DB_{category}_{what}.csv')

    songdb.to_csv(f'YOUR_SONG_DB_{category}_{what}.csv')
    return songdb


In [7]:
# c = input('In which category do you want to search?')
# s = input('What do you want to search there?')
# filldb(c, s.split(','))

# album, artist, track, year, genre
# example: data = filldb('artist', ['Metallica'], 50, 0, 100)

In [8]:
def get_arts_per_category(category, search, limit=50, offset=0):
    arts = sp.search(q=f'{category}:{search}', type='artist', limit=limit, offset=offset)['artists']['items']
    print(search, offset)
    artists = [art['name'] for art in arts]
    return artists

In [ ]:
arts = []
for x in range(0,1000,50):
    try:
        arts.append([get_arts_per_category('year', i, offset=x) for i in range(1950,2024)])
    except:
        print('error')
        continue

In [ ]:
flat_list = [art for sublist in arts for art in sublist]
flat_list = [art for sublist in flat_list for art in sublist]
flat_list = list(set(flat_list))

with open('artists.txt', 'w') as file:
    for art in flat_list:
        file.write(f'{art}\n')
len(flat_list)

In [147]:
%%time

for o in range(0,200,50):
    try:
        filldb('artist', flat_list, offset=o, savepoints=10)
    except Exception as e:
        with open('spotilogger.log', 'a') as logfile:
            logfile.write(f'some thing in the programm went wrong {e}')